In [1]:
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
import re
import requests
import seaborn as sns

from db import create_connection
from db import (
    articles,
    services,
    urls
)

In [2]:
import os
current_path = os.path.abspath(os.getcwd())
datasets_directory = os.path.join(current_path, 'datasets')

if not os.path.exists(datasets_directory):
    os.makedirs(datasets_directory)

In [3]:
connection = create_connection()

# Get data

In [4]:
data = articles.get_articles(connection)

In [5]:
pd.set_option('max_colwidth', 100)
df = pd.DataFrame(data, columns = ['publication_date',
                                   'author',
                                   'title',
                                   'url',
                                   'koronawirus_in_text',
                                   'koronawirus_in_title',
                                   'question_mark',
                                   'exclamation_mark',
                                   'all_words'])

In [6]:
df.sample(frac = 0.001)

,publication_date,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words
7751,NaT,None,None,https://www.se.pl/lublin/dwa-brzegi-program-wydarzenia-atrakcje-bilety-artysci-swieto-filmu-nad-...,NaN,NaN,NaN,NaN,NaN
1165,NaT,None,None,https://www.se.pl/lublin/wyszedl-z-wiezienia-i-znow-zrobil-to-zonie-i-dzieciom-potwor-aresztowan...,NaN,NaN,NaN,NaN,NaN
4318,NaT,None,None,https://www.se.pl/lublin/lubelscy-krolowie-internetu-ich-tworczosc-sledza-miliony-fanow-znacie-i...,NaN,NaN,NaN,NaN,NaN
7136,NaT,None,None,https://www.se.pl/lublin/jaszczow-stary-mlyn-do-rozbiorki-gmina-szuka-spadkobiercow-zdjecia-aa-P...,NaN,NaN,NaN,NaN,NaN
2739,NaT,None,None,https://www.se.pl/lublin/drzwi-szeroko-otwarte-w-xxvii-lo-w-lublinie-galeria-audio-aa-LKXN-gw55-...,NaN,NaN,NaN,NaN,NaN
3952,NaT,None,None,https://www.se.pl/lublin/lubelskie-ponad-950-wypadkow-w-rolnictwie-od-poczatku-roku-zmarlo-juz-5...,NaN,NaN,NaN,NaN,NaN
7798,NaT,None,None,https://www.se.pl/lublin/wyzwanie-smaku-pepsi-i-eska-summer-city-bawia-sie-w-lublinie-aa-7K5t-Zw...,NaN,NaN,NaN,NaN,NaN
1138,NaT,None,None,https://www.se.pl/lublin/janow-lubelski-wjechala-w-kobiete-i-dwojke-dzieci-grozny-wypadek-na-pas...,NaN,NaN,NaN,NaN,NaN
4338,NaT,None,None,https://www.se.pl/lublin/lubelskie-tragiczny-final-pozaru-nie-zyje-58-letni-mezczyzna-aa-EcoQ-Uy...,NaN,NaN,NaN,NaN,NaN


# Clean data

### initial data check, organize and clearing

In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8587 entries, 0 to 8586
Data columns (total 9 columns):
publication_date        1651 non-null datetime64[ns]
author                  1651 non-null object
title                   1651 non-null object
url                     8587 non-null object
koronawirus_in_text     1651 non-null float64
koronawirus_in_title    1651 non-null float64
question_mark           1651 non-null float64
exclamation_mark        1651 non-null float64
all_words               1651 non-null float64
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 2.6 MB


In [8]:
df.nunique()

publication_date        1641
author                   126
title                   1648
url                     8587
koronawirus_in_text       35
koronawirus_in_title       2
question_mark             18
exclamation_mark          15
all_words                383
dtype: int64

In [9]:
df.dropna(inplace=True)
df.shape

(1651, 9)

In [10]:
df['author'] = df['author'].astype('category')
df['koronawirus_in_text'] = df['koronawirus_in_text'].astype('int')
df['koronawirus_in_title'] = df['koronawirus_in_title'].astype('int')
df['question_mark'] = df['question_mark'].astype('int')
df['exclamation_mark'] = df['exclamation_mark'].astype('int')
df['all_words'] = df['all_words'].astype('int')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 8586
Data columns (total 9 columns):
publication_date        1651 non-null datetime64[ns]
author                  1651 non-null category
title                   1651 non-null object
url                     1651 non-null object
koronawirus_in_text     1651 non-null int64
koronawirus_in_title    1651 non-null int64
question_mark           1651 non-null int64
exclamation_mark        1651 non-null int64
all_words               1651 non-null int64
dtypes: category(1), datetime64[ns](1), int64(5), object(2)
memory usage: 765.5 KB


In [11]:
df.count()

publication_date        1651
author                  1651
title                   1651
url                     1651
koronawirus_in_text     1651
koronawirus_in_title    1651
question_mark           1651
exclamation_mark        1651
all_words               1651
dtype: int64

### column: publication_day

In [12]:
df.insert(loc=1, column='publication_day', value=df['publication_date'].dt.strftime('%Y-%m-%d'))
df.sample(frac = 0.002)

,publication_date,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words
902,2020-01-30 09:19:00,2020-01-30,Olka Mazur,Lubelska policja NAJSKUTECZNIEJSZA w kraju. Interwencje co trzy minuty [AUDIO],https://www.se.pl/lublin/lubelskie-najwieksza-wykrywalnosc-przestepstw-w-calej-polsce-audio-aa-r...,0,0,0,0,390
8409,2020-03-20 09:33:00,2020-03-20,mt,Pies ZAATAKOWAŁ 7-latkę! Nie był szczepiony. SCENY GROZY w Puławach,https://www.se.pl/lublin/pies-zaatakowal-7-latke-nie-byl-szczepiony-sceny-grozy-w-pulawach-aa-ze...,0,0,0,2,153
771,2020-03-23 14:02:00,2020-03-23,Mucha,Pozwoliła zabić byłego kochanka,https://www.se.pl/lublin/pozwolila-zabic-bylego-kochanka-aa-Lvte-VSTG-T7up.html,0,0,0,1,228


In [13]:
df.publication_day.value_counts().head()

2020-06-29    28
2020-07-08    26
2020-07-01    25
2020-07-03    22
2020-08-25    22
Name: publication_day, dtype: int64

In [14]:
df['publication_day'].nunique()

233

In [15]:
#df['publication_day'] = df['publication_day'].astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 8586
Data columns (total 10 columns):
publication_date        1651 non-null datetime64[ns]
publication_day         1651 non-null object
author                  1651 non-null category
title                   1651 non-null object
url                     1651 non-null object
koronawirus_in_text     1651 non-null int64
koronawirus_in_title    1651 non-null int64
question_mark           1651 non-null int64
exclamation_mark        1651 non-null int64
all_words               1651 non-null int64
dtypes: category(1), datetime64[ns](1), int64(5), object(3)
memory usage: 873.5 KB


### column: publication_month

In [16]:
df.insert(loc=1, column='publication_month', value=df['publication_date'].dt.strftime('%Y-%m'))
df.sample(frac = 0.001)

,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words
612,2020-05-18 09:33:00,2020-05,2020-05-18,Mucha,"Zrobił tak, jak zrobili jemu. Nietypowe początki młodego oszusta",https://www.se.pl/lublin/zrobil-tak-jak-zrobili-jemu-nietypowe-poczatki-mlodego-oszusta-aa-5YYe-...,0,0,1,0,261
7902,2020-08-27 10:05:00,2020-08,2020-08-27,Marek Targoński,Biała Podlaska: Niekończąca się kwarantanna pani Dominiki. Od 50 dni jest UWIĘZIONA w domu,https://lublin.se.pl/biala-podlaska-niekonczaca-sie-kwarantanna-pani-dominiki-od-50-dni-jest-uwi...,9,1,0,0,282


In [17]:
df.publication_month.value_counts().head(8)

2020-07    439
2020-08    301
2020-06    214
2020-01    154
2020-05    153
2020-03    147
2020-04    138
2020-02    105
Name: publication_month, dtype: int64

In [18]:
df['publication_month'].nunique()

8

In [19]:
#df['publication_month'] = df['publication_month'].astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 8586
Data columns (total 11 columns):
publication_date        1651 non-null datetime64[ns]
publication_month       1651 non-null object
publication_day         1651 non-null object
author                  1651 non-null category
title                   1651 non-null object
url                     1651 non-null object
koronawirus_in_text     1651 non-null int64
koronawirus_in_title    1651 non-null int64
question_mark           1651 non-null int64
exclamation_mark        1651 non-null int64
all_words               1651 non-null int64
dtypes: category(1), datetime64[ns](1), int64(5), object(4)
memory usage: 976.7 KB


### column: author

In [20]:
df['author'].value_counts()[:60]

mt                                 335
Marek Targoński                    183
Mucha                              147
Karolina Januszek                  131
Mariusz Mucha                      111
MTA                                 80
Mucha, mta                          55
AP                                  36
KM                                  33
OM                                  32
Bartłomiej Ważny                    23
Monika Kowalewicz                   22
AC                                  21
Michał Michalak                     19
EIB                                 19
Emilia Białecka                     16
MK                                  16
Mateusz Kasiak                      16
Agnieszka Niećko                    15
Jacek Werner                        13
gk                                  13
Olka Mazur                          13
Artykuł sponsorowany                13
Sylwia Sitka-Czerniak               13
Grzegorz Kluczyński                 13
maal                     

In [21]:
df['author'] = df['author'].str.lower()
regex_pattern = re.compile(r'.*mt.*', re.I)
df[df['author'].str.contains(regex_pattern)].count()

publication_date        514
publication_month       514
publication_day         514
author                  514
title                   514
url                     514
koronawirus_in_text     514
koronawirus_in_title    514
question_mark           514
exclamation_mark        514
all_words               514
dtype: int64

In [22]:
df['author'] = df['author'].replace(['mt', 'mt; wideo: Tygodnik Zamojski', 'gał'],'marek targoński')
# df['author'] = df['author'].replace(['Mucha', 'mucha'],'Mariusz Mucha')
# df['author'] = df['author'].replace(['ŁT'],'Łukasz Trybulski')
# df['author'] = df['author'].replace(['Mateusz Kasiak (Radio Eska)'],'Mateusz Kasiak')
# df['author'] = df['author'].replace(['KaJa'],'Karolina Januszek')
# df['author'] = df['author'].replace(['RS'],'Rafał Strzelec')
# df['author'] = df['author'].replace(['gk'],'Grzegorz Kluczyński')
# df['author'] = df['author'].replace([''],'Nieznany')

# regex_pattern = re.compile(r'(.*,.*|.*/.*|Redakcja ESKA INFO|Nieznany|Akcja partnerska)', re.I)
# df['author'] = df['author'].replace(regex_pattern,'Więcej autorów lub nieznany')

regex_pattern = re.compile(r'.*mt.*', re.I)
df['author'] = df['author'].replace(regex_pattern,'marek targoński')

df['author'].value_counts()[:5]

marek targoński      699
mucha                148
karolina januszek    131
mariusz mucha        111
ap                    36
Name: author, dtype: int64

In [23]:
df['author'] = df['author'].astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651 entries, 0 to 8586
Data columns (total 11 columns):
publication_date        1651 non-null datetime64[ns]
publication_month       1651 non-null object
publication_day         1651 non-null object
author                  1651 non-null category
title                   1651 non-null object
url                     1651 non-null object
koronawirus_in_text     1651 non-null int64
koronawirus_in_title    1651 non-null int64
question_mark           1651 non-null int64
exclamation_mark        1651 non-null int64
all_words               1651 non-null int64
dtypes: category(1), datetime64[ns](1), int64(5), object(4)
memory usage: 971.3 KB


### column: koronawiorus_text

In [24]:
# df.loc[[919, 1979, 1965], 'koronawirus_in_text'] = 0

### column: koronawiorus_anywhere_sum

In [25]:
df['koronawirus_anywhere'] = 0
df['koronawirus_anywhere'] = df['koronawirus_in_text'] + df['koronawirus_in_title']
# df.loc[df.koronawirus_anywhere > 0, 'koronawirus_anywhere'] = 1
df.head(1)

,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words,koronawirus_anywhere
0,2020-08-30 14:52:00,2020-08,2020-08-30,kaja,Salmonella w kurczakach! Główny Inspektorat Sanitarny ostrzega,https://lublin.se.pl/salmonella-w-kurczakach-glowny-inspektorat-sanitarny-ostrzega-ak-zuvx-nuNm-...,2,0,1,0,263,2


### column: koronawiorus_anywhere_count

In [26]:
df['koronawirus_anywhere_count'] = 0
df['koronawirus_anywhere_count'] = (df['koronawirus_in_text'] + df['koronawirus_in_title']) / (df['koronawirus_in_text'] + df['koronawirus_in_title'])
df['koronawirus_anywhere_count'] = df['koronawirus_anywhere_count'].fillna(0).astype('int')
df.head(3)

,publication_date,publication_month,publication_day,author,title,url,koronawirus_in_text,koronawirus_in_title,question_mark,exclamation_mark,all_words,koronawirus_anywhere,koronawirus_anywhere_count
0,2020-08-30 14:52:00,2020-08,2020-08-30,kaja,Salmonella w kurczakach! Główny Inspektorat Sanitarny ostrzega,https://lublin.se.pl/salmonella-w-kurczakach-glowny-inspektorat-sanitarny-ostrzega-ak-zuvx-nuNm-...,2,0,1,0,263,2,1
1,2020-08-28 20:45:00,2020-08,2020-08-28,mariusz mucha,Kiełbasiany ZŁODZIEJ wpadł w Lublinie. Rzucił się Z TULIPANEM na ochroniarza!,https://lublin.se.pl/kielbasiany-zlodziej-wpadl-w-lublinie-aa-Phpp-QAn1-bHnw.html,0,0,0,0,243,0,0
2,2020-08-28 13:43:00,2020-08,2020-08-28,eib,Jaka będzie pogoda w ostatni weekend wakacji? Szykujcie się na najgorsze!,https://lublin.se.pl/jaka-bedzie-pogoda-w-ostatni-weekend-wakacji-szykujcie-sie-na-najgorsze-ak-...,0,0,1,1,171,0,0


### Export data

In [27]:
df.to_csv('datasets/1_clean_data.csv')